<!-- this is for direct deployment. No frontend -->

In [2]:
import cv2
from keras.models import model_from_json
import numpy as np

In [3]:
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)

In [4]:
model.load_weights("emotiondetector.h5")
haar_file=cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade=cv2.CascadeClassifier(haar_file)

In [5]:
def extract_features(image):
    feature = np.array(image)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0

In [7]:
webcam = cv2.VideoCapture(0)
labels = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}

# Check if webcam is opened successfully
if not webcam.isOpened():
    print("Error: Could not open webcam")
    exit()

try:
    while True:
        ret, im = webcam.read()
        if not ret:
            print("Error: Failed to capture frame")
            break
            
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        
        for (p, q, r, s) in faces:
            # Extract face region
            face_roi = gray[q:q+s, p:p+r]
            
            # Draw rectangle around face
            cv2.rectangle(im, (p, q), (p+r, q+s), (255, 0, 0), 2)
            
            # Resize and predict emotion
            face_resized = cv2.resize(face_roi, (48, 48))
            img_features = extract_features(face_resized)
            pred = model.predict(img_features, verbose=0)
            prediction_label = labels[pred.argmax()]
            
            # Add text with better positioning and styling
            cv2.putText(im, prediction_label, (p, q-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        cv2.imshow("Emotion Detection", im)
        
        # Press 'q' to quit or ESC key
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q') or key == 27:
            break

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Clean up resources
    webcam.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 